In [29]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfiguration:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [30]:
from TextSummarization.constants import *
from TextSummarization.utils.common import read_yaml,create_directories

In [31]:
class ConfigurationManager:
    def __init__(
            self,
            config_file_path = CONFIG_FILE_PATH,
            params_file_path = PARAMS_FILE_PATH):

        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)

        create_directories([self.config.artifacts_root])


    def get_data_ingestion_config(self) -> DataIngestionConfiguration:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfiguration(
            root_dir = config.root_dir,
            source_URL = config.source_URL,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )

        return data_ingestion_config

In [32]:
import os
import urllib.request as request
import zipfile
from TextSummarization.logging import logger
from TextSummarization.utils.common import get_size


In [33]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfiguration):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download with \n{headers}")
        else:
            logger.info(f"file already exists of size {get_size(Path(self.config.local_data_file))}")


    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok = True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [34]:
try:
    config = ConfigurationManager()
    data_ingestion_configuration = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_configuration)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-02-08 20:53:13,525: INFO: common: yaml file: config\config.yaml loaded successfully ]
[2024-02-08 20:53:13,525: INFO: common: yaml file: params.yaml loaded successfully ]
[2024-02-08 20:53:13,525: INFO: common: created directory at: artifacts ]
[2024-02-08 20:53:13,543: INFO: common: created directory at: artifacts/data_ingestion ]


[2024-02-08 20:53:37,672: INFO: 321637358: artifacts/data_ingestion/data.zip download with 
Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 0FEA:382E:13F13D:185C27:65C4F1E8
Accept-Ranges: bytes
Date: Thu, 08 Feb 2024 15:23:22 GMT
Via: 1.1 varnish
X-Served-By: cache-maa10235-MAA
X-Cache: MISS
X-Cache-Hits: 0
X-Timer: S1707405802.530937,VS0,VE1162
Vary: Authorization,Accept-Encoding,Origin
Access-Control-Allow-Origin: *
Cross-Origin-Resource-Policy: cross-origin
X-Fastly-Request-ID: 22237dd214812a2533c94352bc2da08e91c4e298
Expires: Thu, 08 Feb 2024 15:28:22 GMT
Source-Age: 1

 ]
